In [20]:
import openpyxl
import xlrd
import docx
from PyPDF2 import PdfReader
import PyPDF2
import fitz
import pytesseract
from PIL import Image
import io

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# OCR function for images
def ocr_image(file_path):
    image = Image.open(file_path)
    return pytesseract.image_to_string(image)

# Function to read text from Excel files
def read_excel(file_path):
    # Handle .xlsx files using openpyxl
    if file_path.endswith('.xlsx'):
        wb = openpyxl.load_workbook(file_path)
        text = ""
        for sheet_name in wb.sheetnames:
            sheet = wb[sheet_name]
            for row in sheet.iter_rows(values_only=True):
                for cell_value in row:
                    text += str(cell_value) + " "
                text += "\n"
        return text
    # Handle .xls files using xlrd
    elif file_path.endswith('.xls'):
        wb = xlrd.open_workbook(file_path)
        text = ""
        for sheet_name in wb.sheet_names():
            sheet = wb.sheet_by_name(sheet_name)
            for row_num in range(sheet.nrows):
                row = sheet.row_values(row_num)
                for cell_value in row:
                    text += str(cell_value) + " "
                text += "\n"
        return text
    else:
        raise ValueError("Unsupported file format for Excel.")

#end

# Function to read text from Word documents
def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

# Function to read text from PDF files
#new code

def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

# Function to read text from PDF files with OCR for scanned images
def read_pdf_with_ocr(file_path):
    text_from_pdf = extract_text_from_pdf(file_path)
    
    # Check if there are any images in the PDF
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        images = []
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            if '/XObject' in page['/Resources']:
                xObject = page['/Resources']['/XObject'].get_object()
                if xObject.get_object() and '/Image' in xObject.get_object():
                    images.append(xObject.get_object()['/Image'])

    text_from_images = ""
    for image in images:
        if image['/ColorSpace'] == '/DeviceRGB':
            image_data = image.get_object()
            img = Image.open(io.BytesIO(image_data)).convert("RGB")
            text_from_images += ocr_image(img)

    return text_from_pdf + text_from_images

#end of code


# Main function to handle different file types
def read_text_from_file(file_path):
    #All types of images
    if file_path.endswith('.jpg'):
        return ocr_image(file_path)
    elif file_path.endswith('.png'):
        return ocr_image(file_path)
    elif file_path.endswith('.jpeg'):
        return ocr_image(file_path)
    elif file_path.endswith('.gif'):
        return ocr_image(file_path) 
    elif file_path.endswith('.xlsx'):
        return read_excel(file_path)
    elif file_path.endswith('.xls'):
        # Handle .xls files using xlrd library
        return read_excel(file_path)
    elif file_path.endswith('.docx'):
        return read_word(file_path)
    elif file_path.endswith('.pdf'):
        # If PDF contains images, use OCR
        return read_pdf_with_ocr(file_path)
    else:
        # For other file types or unsupported formats
        return "Unsupported file format."

# Take input from the user
file_path = input("Import your File: ")
result = read_text_from_file(file_path)
print(result)

Import your File: a.jpg


FileNotFoundError: [Errno 2] No such file or directory: 'a.jpg'

In [25]:
import pytesseract
from PIL import Image
import json

def ocr_image(image_path):
    try:
        # Open the image using Pillow (PIL)
        image = Image.open(image_path)

        # Use pytesseract to perform OCR on the image
        text = pytesseract.image_to_string(image)

        # Create a dictionary to hold the extracted data
        extracted_data = {"text": text}

        # Convert the dictionary to JSON
        json_data = json.dumps(extracted_data)

        return json_data
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Example usage:
image_path = r'C:\Users\Nasrullah\Desktop\FYP\Data\a.jpg'
json_result = ocr_image(image_path)

if json_result is not None:
    print("Extracted Data in JSON:")
    print(json_result)

Extracted Data in JSON:
{"text": "\\Ne Stoet With Good\n\nBecause all businesses Should\nbe gate pees: bs pee\n"}
